<a href="https://colab.research.google.com/github/Ruran8wa/Sentiment_analysis/blob/main/IMDB_reviews_Sentiment_analysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

**Imports**

In [ ]:
import json

import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

**Loading Data and Splitting them into test and train data**

In [ ]:
data = pd.read_csv('/content/IMDB_Dataset.csv')
print(data.shape)
print(data.head())
print(data.tail())
print(data.isnull().sum())
print(data["sentiment"].value_counts())

(50000, 2)
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
                                                  review sentiment
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative
review       0
sentiment    0
dtype: int64
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [ ]:
data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)
print(data["sentiment"].value_counts())

sentiment
1    25000
0    25000
Name: count, dtype: int64


/tmp/ipython-input-2055439680.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment": {"positive": 1, "negative": 0}}, inplace=True)


In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


**Data Preprocessing**

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

train_data['review'] = train_data['review'].apply(remove_stopwords)
test_data['review'] = test_data['review'].apply(remove_stopwords)

print("Reviews after removing stopwords:")
print(train_data['review'].head())
print(test_data['review'].head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Reviews after removing stopwords:
39087    thats kept asking many fights screaming matche...
30893    watch entire movie could watch entire movie st...
45278    touching love story reminiscent mood love draw...
16398    latterday fulci schlocker totally abysmal conc...
13653    first firmly believe norwegian movies continua...
Name: review, dtype: object
33553    really liked summerslam due look arena curtain...
9427     many television shows appeal quite many differ...
199      film quickly gets major chase scene ever incre...
12447    jane austen would definitely approve onebr br ...
39489    expectations somewhat high went see movie thou...
Name: review, dtype: object


In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])

X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

print(X_train)
print(X_test)

[[   0    0    0 ...  112  231 3646]
 [   0    0    0 ...  104   21   30]
 [   0    0    0 ...  201  605   12]
 ...
 [   0    0    0 ... 1009 1481  516]
 [   0    0    0 ... 1138  143   30]
 [   0    0    0 ...  370   13 1811]]
[[   0    0    0 ...  171   94  849]
 [   0    0    0 ... 1758   11    1]
 [   0    0    0 ...  829  939   22]
 ...
 [   0    0    0 ...  481   92 3183]
 [   0    0    0 ...  118   69 2089]
 [   0    0    0 ...   11  224    4]]


In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

print(Y_train)
print(Y_test)

39087    0
30893    0
45278    1
16398    0
13653    0
        ..
11284    1
44732    1
38158    0
860      1
15795    1
Name: sentiment, Length: 40000, dtype: int64
33553    1
9427     1
199      0
12447    1
39489    0
        ..
28567    0
25079    1
18707    1
15200    0
5857     1
Name: sentiment, Length: 10000, dtype: int64
